In [ ]:
# Required for importing modules from parent directory
import os
import sys

current_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.dirname(current_dir)
print(parent_dir)
sys.path.append(parent_dir)

In [ ]:
from pathlib import Path

import soccer_twos

from src.agents.ddpg_agent import DDPGAgent
from src.agents.ddqn_agent import DDQNAgent
from src.agents.maddpg_agent import MADDPGAgent
from src.config import CHECKPOINT_DIR, N_GAMES
from src.utils import RewardShaper

In [ ]:
def evaluate_agent(agent, n_games=N_GAMES):
    env = soccer_twos.make(
        render=True,
        time_scale=1,
        quality_level=5,
    )
    reward_shaper = RewardShaper()

    for i in range(n_games):
        obs = env.reset()
        done = False
        score = 0
        while not done:
            actions = {
                0: [0, 0, 0],
                1: [0, 0, 0],
                2: [0, 0, 0],
                3: [0, 0, 0],
            }
            actions[0] = agent.act(obs[0])
            print(actions)

            next_obs, reward, done, info = env.step(actions)
            done = done["__all__"]
            obs = next_obs
            score = reward[0] + reward_shaper.calculate_reward(
                obs[0], next_obs[0], info, 0
            )
            env.render()  # Render the environment to visualize the game

        print(f"Evaluation Game {i}, Score: {score}")

In [ ]:
# Load the best agent (example: checkpoint at episode 90)
best_agent = MADDPGAgent(336, 3)
best_agent.load(parent_dir / Path(CHECKPOINT_DIR) / "checkpoint_DDQN_agent_1_40.pth")

# Evaluate the trained agent
evaluate_agent(best_agent, n_games=5)